## 🧪 Analyst Toolkit Tutorial: Full Data Pipeline

This interactive notebook demonstrates the complete analyst pipeline using a synthetic **Palmer Penguins** dataset.

Each step in the pipeline is modular, YAML-configurable, and produces exports, plots, and certification-ready reports.

### 🧰 Toolkit Architecture: 3-Way Modular Design

This pipeline is built around a flexible ETL framework with three usage modes:

- 📓 **Notebook Mode**: Run individual modules or the full pipeline interactively. Ideal for exploration and QA.
- 🧵 **CLI Mode**: Execute the full pipeline using `run_toolkit_pipeline.py`, controlled via a master YAML config.
- 🧪 **Hybrid Mode**: Develop in notebooks, deploy via scripts, reusing the same configs.

In [1]:
# 📁 1. Load Configuration and Set Execution Context

import os
from pathlib import Path
from analyst_toolkit.m00_utils.config_loader import load_config
from analyst_toolkit.m00_utils.load_data import load_csv
from analyst_toolkit.m01_diagnostics.run_diag_pipeline import run_diag_pipeline
from analyst_toolkit.m02_validation.run_validation_pipeline import run_validation_pipeline
from analyst_toolkit.m03_normalization.run_normalization_pipeline import run_normalization_pipeline
from analyst_toolkit.m04_duplicates.run_dupes_pipeline import run_duplicates_pipeline
from analyst_toolkit.m05_detect_outliers.run_detection_pipeline import run_outlier_detection_pipeline
from analyst_toolkit.m06_outlier_handling.run_handling_pipeline import run_outlier_handling_pipeline
from analyst_toolkit.m07_imputation.run_imputation_pipeline import run_imputation_pipeline
from analyst_toolkit.m10_final_audit.final_audit_pipeline import run_final_audit_pipeline

# --- Find Project Root ---
# This helper function makes the notebook runnable from any subdirectory
# by locating the project root based on a set of marker directories.
def find_project_root(markers=("config", "notebooks", "data")):
    """Searches upward from the current directory for marker directories to find the project root."""
    current_path = Path.cwd().resolve()
    for parent in [current_path, *current_path.parents]:
        if all((parent / marker).is_dir() for marker in markers):
            return parent
    # Fallback to current working directory if no marker is found
    print(f"⚠️ Could not find project root with markers {markers}. Using current directory.")
    return Path.cwd()

# --- Set Project Root as Working Directory ---
PROJECT_ROOT = find_project_root()
os.chdir(PROJECT_ROOT)
# This print statement now only shows the project folder name, not the full path.
print(f"📂 Project Root set to: '{PROJECT_ROOT.name}'")

# --- Load Master Config ---
# Path to master config is now relative to the project root
master_config_path = "config/run_toolkit_config.yaml"

# Load master configuration dictionary
master_config = load_config(master_config_path) # load_config will resolve the relative path
run_id = master_config.get("run_id", "default_run")
notebook_mode = master_config.get("notebook", True)

print(f"🔧 Config loaded | Run ID: {run_id} | Notebook Mode: {notebook_mode}")


📂 Project Root set to: 'dirty_birds_eda_3'
🔧 Config loaded | Run ID: demo_run_02 | Notebook Mode: True


In [2]:
# 📥 2. Load Raw Data

# Load input path from the master config
relative_input_path = master_config.get("pipeline_entry_path")
if not relative_input_path:
    raise ValueError("❌ 'pipeline_entry_path' not found in master config.")

# Since the working directory is the project root, we can use the relative path directly.
print(f"📂 Loading data from: {relative_input_path}")
df_raw = load_csv(relative_input_path)

📂 Loading data from: data/raw/synthetic_penguins_v0.4.0.csv


## 📊 M01 — Diagnostics

This module generates a profile of the raw data: shape, types, nulls, skewness, and sample rows. It's the first step in understanding your dataset's structure and quality.

In [ ]:
# --- Run Diagnostics Module ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("diagnostics", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Diagnostics module as per master config.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/diag_config.yaml")
    # The path is now relative to the project root"
    diag_config = load_config(relative_config_path)
    print("🚀 Running Diagnostics from '{relative_config_path}'...")

    df_profiled = run_diag_pipeline(
        config=diag_config,
        df=df_raw,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Diagnostics from '{relative_config_path}'...


Metric,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
bill_length_mm,5326.0,45.249156,5.496079,31.92,40.6325,46.05,49.37,63.60,-0.176468,-0.695710
bill_depth_mm,5334.0,17.290041,2.208216,12.75,15.4900,17.50,19.05,22.62,-0.147417,-0.959897
flipper_length_mm,5300.0,201.724050,13.958287,158.21,190.9000,198.90,214.00,244.39,0.238775,-0.832208
body_mass_g,5325.0,3829.834656,835.186271,2389.96,3229.0000,3729.00,4317.83,6637.71,0.503893,-0.284733


tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg
NaN,Gentoo,48.990000,14.63,220.9,5890.000000,Adult,Male,Torgersen North,Torgersen,2023-11-17,Healthy,PAPRI2023,Yes,2023-11-09
ADE-0001,Adelie,39.550000,19.92,186.2,2500.000000,Chick,Male,Biscoe West,NaN,2022-07-31,Underweight,PAPRI2022,Yes,2022-07-20
NaN,Gentoo,55.734313,13.00,NaN,4536.000000,Adult,Female,Biscoe West,Biscoe,2024-04-14,Healthy,NaN,Yes,2024-04-12
GEN-0001,Gentoo,46.220000,13.91,212.8,3150.000000,chik,NaN,Dream South,Dream,2020-04-27,Underweight,PAPRI2020,Yes,2020-04-14
NaN,Chinstrap,49.020000,16.22,192.2,3120.389387,ADLT,NaN,Biscoe West,Biscoe,2022-10-03,Healthy,PAPRI2022,Yes,2022-10-02


Accordion(children=(VBox(children=(HTML(value="<h3 style='margin-top:10px'>Visual Profile</h3>"), HBox(childre…

## 🛡️ M02 — Validation (Audit Mode)

This module audits the dataset against a defined schema to catch issues early and guide cleaning steps:

- **Expected Columns & Dtypes**
- **Allowed Categorical Values**
- **Numeric Range Checks**

In this first pass, `fail_on_error` is `false`, so it reports all issues without halting the pipeline.

In [ ]:
# --- Run Validation Module ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("validation", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Outlier Detection module as per master config.")
elif df_profiled is None:
    print("⏩ Skipping Outlier Detection because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/validation_config.yaml")
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Validation from '{relative_config_path}'...")
    # run_validation_pipeline returns (df, results_dict)
    df_valid = run_validation_pipeline(
        config=module_config,
        df=df_profiled,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Validation from 'config/validation_config_autofill.yaml'...


Validation Rule,Description,Status
Schema Conformity,Verify column names match the expected schema.,✅ Pass
Dtype Enforcement,Verify column data types match expectations.,⚠️ Fail (2 issues)
Categorical Values,Verify values in categorical columns are within an allowed set.,⚠️ Fail (7 issues)
Numeric Ranges,Verify values in numeric columns are within a defined range.,⚠️ Fail (4 issues)


## 🧹 M03 — Normalization

This module performs rule-based cleaning and standardization to prepare the dataset for certification:

- **Column Renaming & Type Coercion**
- **Value Mapping & Text Cleaning**
- **Fuzzy Matching & Datetime Parsing**

All rules and output paths are controlled via the YAML config (`normalization_config_template.yaml`).

In [ ]:
# --- Run Normalization Module ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("normalization", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Outlier Detection module as per master config.")
elif df_valid is None:
    print("⏩ Skipping Outlier Detection because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/normalization_config.yaml")
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Normalization from '{relative_config_path}'...")
    df_norm= run_normalization_pipeline(
        config=module_config,
        df=df_valid,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Normalization from 'config/normalization_config.yaml'...


Original Name,New Name
tag_id,tag_id
species,species
bill_length_mm,bill_length_mm
bill_depth_mm,bill_depth_mm
flipper_length_mm,flipper_length_mm
body_mass_g,body_mass_g
age_group,age_group
sex,sex
colony_id,colony_id
island,island


## 🛡️ M02 — Certification Gate (Strict Mode)

This step re-uses the **Validation Module (M02)**, but with a stricter configuration to act as a quality gate. It is designed to **halt the pipeline** if violations are found:

- ✅ All column names, data types, categorical values, and numeric ranges must pass
- 🛑 **`fail_on_error: true`** triggers a hard stop on validation failure

This step certifies the cleaned dataset before proceeding to more advanced steps like outlier handling.

In [ ]:
# --- Run Certification Gate (Strict Validation) ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("validation_gatekeeper", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Outlier Detection module as per master config.")
elif df_norm is None:
    print("⏩ Skipping Outlier Detection because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/certification_config.yaml")
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Certification Gate from '{relative_config_path}'...")
    df_cert = run_validation_pipeline(
        config=module_config,
        df=df_norm,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Certification Gate from 'config/certification_config_template.yaml'...


Validation Rule,Description,Status
Schema Conformity,Verify column names match the expected schema.,✅ Pass
Dtype Enforcement,Verify column data types match expectations.,✅ Pass
Categorical Values,Verify values in categorical columns are within an allowed set.,✅ Pass
Numeric Ranges,Verify values in numeric columns are within a defined range.,⚠️ Fail (4 issues)


tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg
ADE-0095,adelie,31.92,19.28,201.24,4033.01,adult,male,torgersen north,torgersen,2022-12-14,underweight,papri2020,yes,2020-04-18
CHN-0362,chinstrap,62.16,18.53,197.26,2674.45,adult,NaN,biscoe west,cormorant,2024-12-22,NaN,papri2023,yes,2023-07-19
CHN-0852,chinstrap,63.60,17.50,203.48,3494.88,adult,female,torgersen north,torgersen,2023-08-24,healthy,papri2023,yes,2023-08-10
tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg
ADE-0336,adelie,40.95,21.84,186.52,3707.87,adult,female,biscoe west,biscoe,2024-05-06,healthy,papri2023,yes,2023-05-20
ADE-0816,adelie,38.55,21.45,187.67,2927.74,adult,female,dream south,dream,2024-12-11,critically ill,papri2024,yes,2024-01-03
CHN-0976,chinstrap,52.57,21.51,NaN,3333.46,adult,NaN,cormorant east,cormorant,2024-12-12,healthy,papri2023,no,NaT
ADE-0339,adelie,40.53,21.54,NaN,2596.45,juvenile,NaN,dream south,dream,NaT,underweight,NaN,yes,2021-02-19
CHN-0339,chinstrap,48.68,21.59,202.71,NaN,adult,NaN,torgersen north,torgersen,2024-05-15,healthy,papri2021,yes,2021-09-29
tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg


## ♻️ M04 — Deduplication

This module identifies and handles **duplicate rows** in the dataset, using the logic from `m04_duplicates`.

You can choose to:
- 🔍 **Flag duplicates** for review (`mode: "flag"`)
- ✂️ **Remove duplicates** directly (`mode: "remove"`)

The logic is configurable via `dups_config_template.yaml`, allowing you to specify which columns to check for duplication.

In [ ]:
# --- Run Duplicates Module ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("duplicates", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Outlier Detection module as per master config.")
elif df_cert is None:
    print("⏩ Skipping Outlier Detection because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/dups_config.yaml")
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Duplicates from '{relative_config_path}'...")
    # The function returns the dataframe and a results dictionary
    df_duped = run_duplicates_pipeline(
        config=module_config,
        df=df_cert,
        notebook=notebook_mode,
        run_id=run_id
    )


🚀 Running Duplicates from 'config/dups_config.yaml'...


Accordion(children=(VBox(children=(HTML(value="<h3 style='margin-top:10px'>Visual Summary</h3>"), HBox(childre…

## 📏 M05 — Detect Outliers

This module (`m05_detect_outliers`) scans numeric columns for outliers using configurable logic:

- **Z-Score** or **IQR** methods (per column or global default)
- Adds binary flags (e.g., `*_outlier`) to the dataset if `append_flags: true`
- Skips non-numeric or excluded fields via `exclude_columns`

📊 If enabled, an interactive **PlotViewer** renders boxplots, histograms, and violin plots inline, giving a fast visual summary of where anomalies occur.

In [ ]:
# --- Detect Outliers ---

# Initialize detection_results to ensure it exists for the next step
detection_results = None

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("outlier_detection", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Outlier Detection module as per master config.")
elif df_duped is None:
    print("⏩ Skipping Outlier Detection because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/outlier_config.yaml")
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Outlier Detection from '{relative_config_path}'...")
    # This function returns (df_with_flags, detection_results_dict)
    df_detect, detection_results = run_outlier_detection_pipeline(
        config=module_config,
        df=df_duped,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Outlier Detection from 'config/outlier_config_template.yaml'...


column,method,outlier_count,lower_bound,upper_bound,outlier_examples
bill_length_mm,iqr,1,27.52625,62.47625,[63.6]


tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg,is_duplicate
CHN-0852,chinstrap,63.6,17.5,203.48,3494.88,adult,female,torgersen north,torgersen,2023-08-24,healthy,papri2023,yes,2023-08-10,True


Accordion(children=(VBox(children=(HTML(value="<h3 style='margin-top:10px'>Outlier Visualizations</h3>"), HBox…

## 🧼 M06 — Handle Outliers

This module (`m06_outlier_handling`) applies cleanup strategies to outliers flagged in the detection step:

- **Strategies**: `clip` (cap to bounds), `median` (impute), `constant` (fill with a fixed value), or `none`.
- **Configuration**: Apply rules globally (`__default__`) or per-column.

This step is purely for remediation and relies on the `detection_results` from the previous module.

In [ ]:
# --- Handle Outliers ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("outlier_handling", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Outlier Handling module as per master config.")
elif df_detect is None:
    print("⏩ Skipping Outlier Handling because input dataframe is None.")
elif detection_results is None:
    print("⏩ Skipping Outlier Handling because no detection results were provided from the previous step.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/handling_config.yaml")
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Outlier Handling from '{relative_config_path}'...")
    # This function returns the dataframe with outliers handled
    df_handled= run_outlier_handling_pipeline(
        config=module_config,
        df=df_detect,
        detection_results=detection_results,  # Pass results from M05
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Outlier Handling from 'config/handling_config_template.yaml'...


strategy,column,outliers_handled,details
clip,bill_length_mm,1,Clipped 1 values to bounds.


Column,Row_Index,Original_Value,Capped_Value
bill_length_mm,5760,63.6,62.47625


## 🔧 M07 — Impute Missing Values

This module (`m07_imputation`) fills missing (`NaN`) values using a column-specific strategy:

- **Strategies**: `mean`, `median`, `mode`, or `constant`.
- **Configuration**: Apply rules per column via `rules.strategies` in the YAML.

📊 If enabled, comparison plots show how categorical columns changed post-imputation.

In [ ]:
# --- Run Imputation Module ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("imputation", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Imputation module as per master config.")
elif df_handled is None:
    print("⏩ Skipping Imputation because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/imputation_config.yaml")
    absolute_config_path = PROJECT_ROOT / relative_config_path
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Imputation from '{relative_config_path}'...")
    df_imput = run_imputation_pipeline(
        config=module_config,
        df=df_handled,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Imputation from 'config/imputation_config_template.yaml'...


Column,Strategy,Fill Value,Nulls Filled
bill_length_mm,mean,45.25,447
body_mass_g,mean,3829.83,448
bill_depth_mm,median,17.50,439
flipper_length_mm,median,198.90,473
sex,mode,male,2862
tag_id,constant,UNKNOWN,1689
species,constant,UNKNOWN,171
age_group,constant,UNKNOWN,96
colony_id,constant,UNKNOWN,407
island,constant,UNKNOWN,524


Column,Remaining Nulls
bill_length_mm_iqr_outlier,447


Accordion(children=(VBox(children=(HTML(value="<h3 style='margin-top:10px'>Imputation Visualizations</h3>"), H…

## Alternative: Full Pipeline Runner

For non-interactive runs, or to execute the entire pipeline in one go, you can use the `run_toolkit_pipeline` function from the `analyst_toolkit` library. This is particularly useful for automated scripts where step-by-step inspection is not required.

To use it, you would import `run_toolkit_pipeline` and call it with the path to your master configuration file.

```python
# from analyst_toolkit.run_toolkit_pipeline import run_toolkit_pipeline
#
# This function runs all modules enabled in your 'run_toolkit_config.yaml' sequentially.
# df_final, all_results = run_toolkit_pipeline(config_path=RUN_CONFIG_PATH)
```

> **Note:** This template notebook is designed for step-by-step execution and inspection. Using the full pipeline runner will execute all steps at once and bypass the individual cell outputs in this notebook.

## 🎬 M10 — Final Audit & Certification

This final module (`m10_final_audit`) serves as the ultimate quality gate before exporting the cleaned dataset. It performs a comprehensive audit and applies strict certification checks.

- ✅ **Final Edits**: Drops or renames columns and coerces dtypes as needed.
- ✅ **Certification Check**: Re-runs validation rules with `fail_on_error: true` to enforce schema, dtypes, and content requirements.
- ✅ **Lifecycle Comparison**: Compares the raw vs. final dataset's structure, nulls, and column presence.
- ✅ **Capstone Report**: Renders a complete dashboard summarizing the pipeline's impact and status.

🛡️ If any rule is violated, the system halts and logs failure details for debugging. Once this step passes, your dataset is certified and ready for production use.

In [ ]:
# --- Run Final Audit Module ---

# Check if module is enabled in the master config
module_settings = master_config.get("modules", {}).get("final_audit", {})
if not module_settings.get("run", False):
    print("⏩ Skipping Final Audit module as per master config.")
elif df_imput is None:
    print("⏩ Skipping Final Audit because input dataframe is None.")
else:
    # Load module-specific config
    relative_config_path = module_settings.get("config_path", "config/final_audit_config.yaml")
    absolute_config_path = PROJECT_ROOT / relative_config_path
    module_config = load_config(relative_config_path)
    
    print(f"🚀 Running Final Audit from '{relative_config_path}'...")
    # This function returns the final, certified dataframe
    df_final = run_final_audit_pipeline(
        config=module_config,
        df=df_imput,
        notebook=notebook_mode,
        run_id=run_id
    )

🚀 Running Final Audit from 'config/final_audit_config_template.yaml'...


Issue Type,Columns
Unexpected,"sex, colony_id, health_status, capture_date, study_name, flipper_length_mm, tag_id, age_group, bill_depth_mm, clutch_completion, date_egg, body_mass_g, is_duplicate, island, bill_length_mm, species"


Metric,Value
Final Pipeline Status,❌ CERTIFICATION FAILED
Certification Rules Passed,False
Null Value Audit Passed,True
Action,Details
drop_columns,Removed: ['bill_length_mm_iqr_outlier']


Metric,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
bill_length_mm,5773.0,45.248945,5.278319,31.92,41.14,45.4,49.11,62.47625,-0.184926,-0.506671
bill_depth_mm,5773.0,17.306007,2.123310,12.75,15.65,17.5,18.93,22.62000,-0.175695,-0.789962
flipper_length_mm,5773.0,201.492667,13.396554,158.21,191.70,198.9,212.80,244.39000,0.299019,-0.635365
body_mass_g,5773.0,3829.834656,802.119748,2389.96,3272.35,3806.0,4231.00,6637.71000,0.524650,-0.056235


tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg,is_duplicate
UNKNOWN,gentoo,48.990000,14.63,220.9,5890.000000,adult,male,torgersen north,torgersen,2023-11-17,healthy,papri2023,yes,2023-11-09,False
ADE-0001,adelie,39.550000,19.92,186.2,2500.000000,chick,male,biscoe west,UNKNOWN,2022-07-31,underweight,papri2022,yes,2022-07-20,False
UNKNOWN,gentoo,55.734313,13.00,198.9,4536.000000,adult,female,biscoe west,biscoe,2024-04-14,healthy,UNKNOWN,yes,2024-04-12,False
GEN-0001,gentoo,46.220000,13.91,212.8,3150.000000,chick,male,dream south,dream,2020-04-27,underweight,papri2020,yes,2020-04-14,False
UNKNOWN,chinstrap,49.020000,16.22,192.2,3120.389387,adult,male,biscoe west,biscoe,2022-10-03,healthy,papri2022,yes,2022-10-02,False


In [12]:
# 🎉 Final Certified Data Preview
if 'df_final' in locals() and df_final is not None:
    print("✅ Pipeline complete. Displaying the first 5 rows of the final certified dataset:")
    display(df_final.head())
else:
    print("⏹️ Pipeline finished, but no final dataframe was produced (likely skipped or failed). ")

✅ Pipeline complete. Displaying the first 5 rows of the final certified dataset:


,tag_id,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,age_group,sex,colony_id,island,capture_date,health_status,study_name,clutch_completion,date_egg,is_duplicate
0,UNKNOWN,gentoo,48.990000,14.63,220.9,5890.000000,adult,male,torgersen north,torgersen,2023-11-17,healthy,papri2023,yes,2023-11-09,False
1,ADE-0001,adelie,39.550000,19.92,186.2,2500.000000,chick,male,biscoe west,UNKNOWN,2022-07-31,underweight,papri2022,yes,2022-07-20,False
2,UNKNOWN,gentoo,55.734313,13.00,198.9,4536.000000,adult,female,biscoe west,biscoe,2024-04-14,healthy,UNKNOWN,yes,2024-04-12,False
3,GEN-0001,gentoo,46.220000,13.91,212.8,3150.000000,chick,male,dream south,dream,2020-04-27,underweight,papri2020,yes,2020-04-14,False
4,UNKNOWN,chinstrap,49.020000,16.22,192.2,3120.389387,adult,male,biscoe west,biscoe,2022-10-03,healthy,papri2022,yes,2022-10-02,False
